## Cleaning Data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('murder_data.csv')

In [3]:
print(data.columns)
data.head(1)

Index(['ID', 'CNTYFIPS', 'Ori', 'State', 'Agency', 'Agentype', 'Source',
       'Solved', 'Year', 'StateName', 'Month', 'Incident', 'ActionType',
       'Homicide', 'Situation', 'VicAge', 'VicSex', 'VicRace', 'VicEthnic',
       'OffAge', 'OffSex', 'OffRace', 'OffEthnic', 'Weapon', 'Relationship',
       'Circumstance', 'Subcircum', 'VicCount', 'OffCount', 'FileDate',
       'fstate', 'MSA'],
      dtype='object')


,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,197601001AKASP00,"Juneau, AK",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,ALA,...,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180,Alabama,"Birmingham-Hoover, AL"


In [5]:

print('% of Unknown VicEthnic Entries: ' + str(round(data.ID.loc[data['VicEthnic']=='Unknown or not reported'].count()/data.ID.count()*100))+'%')
print('% of Unknown OffEthnic Entries: ' + str(round(data.ID.loc[data['OffEthnic']=='Unknown or not reported'].count()/data.ID.count()*100))+'%')
print('% of VicCount = 0 Entries: ' + str(round(data.ID.loc[data['VicCount']==0].count()/data.ID.count()*100))+'%')
print('% of OffCount = 0 Entries: ' + str(round(data.ID.loc[data['OffCount']==0].count()/data.ID.count()*100))+'%')

% of Unknown VicEthnic Entries: 62.0%
% of Unknown OffEthnic Entries: 73.0%
% of VicCount = 0 Entries: 92.0%
% of OffCount = 0 Entries: 88.0%


In [6]:
data = data.drop(['VicEthnic','OffEthnic','VicCount','OffCount'],1)

In [7]:
data.columns

Index(['ID', 'CNTYFIPS', 'Ori', 'State', 'Agency', 'Agentype', 'Source',
       'Solved', 'Year', 'StateName', 'Month', 'Incident', 'ActionType',
       'Homicide', 'Situation', 'VicAge', 'VicSex', 'VicRace', 'OffAge',
       'OffSex', 'OffRace', 'Weapon', 'Relationship', 'Circumstance',
       'Subcircum', 'FileDate', 'fstate', 'MSA'],
      dtype='object')

In [8]:
# Changes name of ID column to CaseID to prevent issues when creating sqlite database
data = data.rename(columns={'ID':'CaseID'})

In [9]:
# Changes VicAge string values to integers
data['VicAge'] = data['VicAge'].replace('Newborn or infant', 1)
data['VicAge'] = data['VicAge'].replace('Age unknown', 0)
data['VicAge'] = data['VicAge'].replace('99 years old and more', 99)
# Changes OffAge string values to integers
data['OffAge'] = data['OffAge'].replace('Offender unknown', 0)
data['OffAge'] = data['OffAge'].replace('99 years old and more', 99)

In [10]:
# Rhode Island was spelled incorrectly in FBI database...go figure.
data['State'] = data['State'].replace('Rhodes Island','Rhode Island')
data.loc[data['State']=='Rhode Island'].head(1)

,CaseID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,...,OffAge,OffSex,OffRace,Weapon,Relationship,Circumstance,Subcircum,FileDate,fstate,MSA
604157,200708001CA03710,"San Diego, CA",RI00409,Rhode Island,Providence,Municipal police,FBI,Yes,1976,R I,...,22,Male,White,"Firearm, type not stated",Stranger,Other arguments,NaN,30180,Rhode Island,"Providence-New Bedford-Fall River, RI-MA"


In [11]:
# Rewrites data to new CSV file
data.to_csv('clean_murder_data.csv', index=False)

## Add State Abbreviations column

In [ ]:
stateAbbr = pd.read_csv("statesAbbr.csv")

In [ ]:
# This function outputs a State's abbreviation based on State's name as input value
def state_to_stateAbbr(state):
    d = stateAbbr.set_index('State').T.to_dict('records')
    s = state.strip()
    try:
        abbr=d[0][s]
        return abbr
    except:
        raise ValueError('ERROR')

In [ ]:
# Creates a new column for the State's abbreviation based on 'State' column
# Added a progress bar using tqdm library
from tqdm import tqdm
tqdm.pandas(desc='This is taking FOREVER!!!')
data['StateAbbr'] = data['State'].progress_map(state_to_stateAbbr)